In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import string
import time
import sqlite3
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [12]:
# global variables

# used to create id strings later
base_url = 'https://www.basketball-reference.com'

season_gamecount = 1

precovid_seasons = ['0304','0405', '0506', '0607', '0708','0809', '0910', '1011', '1112', '1213']
precovid_url_years = ['2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']

# used to create sql database table columns
info_columns = ['game_id', 'season', 'date', 'away_team', 'away_score', 'home_team', 'home_score', 'result']
num_columns = ['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
               'FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
# pause between each server call
delay = time.sleep(np.random.randint(3,6))

In [3]:
def create_game_info(url, season_id, season_gamecount):
    
    game_count = str(season_gamecount)
    while len(game_count) < 4:
        game_count = '0' + game_count
    
    id_string = url.strip(string.ascii_letters+string.punctuation)
    year = id_string[0:4]
    month = id_string[4:6]
    day = id_string[6:8]
    
    date = year+'-'+month+'-'+day
    
    game_id = int(season_id+month+day+game_count)
    season_id = int(season_id)
    
    return [game_id, season_id, date]

In [4]:
def create_team_info(table):
    '''
    Create a dataframe with game results. Uses an html table as input.
    
    ---
    Inputs:
    
    table: a BeautifulSoup html table
    ---
    Outputs:
    
    team_info: a dataframe with the relevant game information (team_ids, scores, and boolean 'results' column)
    '''
    
    # get team_ids
    id_rows = table.findAll('th', attrs={'class':'center', 'data-stat':'team', 'scope':'row'})
    team_ids = [row.text.strip() for row in id_rows]
    
    # get final score
    scores = table.findAll('td', attrs={'class': 'center', 'data-stat': 'T'})
    final_scores = [int(score.text.strip()) for score in scores]
    
    # boolean game-winner: away=0, home=1
    if final_scores[0] > final_scores[1]:
        result=0
    else:
        result=1
    
    team_info = [team_ids[0], final_scores[0], team_ids[1], final_scores[1], result]
    
    return team_info

In [5]:
def create_info_df(game_info, team_info, info_columns):
    info = game_info + team_info
    info_df = pd.DataFrame([info], columns=info_columns)
    return info_df

In [6]:
def create_boxscores(table, game_id):

    # ignore first 'tr', it is table title, not column
    rows = table.findAll('tr')[1:]
    # first 'th' is 'Starters', but will be changed into the player names
    headers = rows[0].findAll('th')
    # provide column names
    headerlist = [h.text.strip() for h in headers]
    
    # ignore first row (headers)
    data = rows[1:]
    # get names column
    player_names = [row.find('th').text.strip() for row in rows]
    # get player stats
    player_stats = [[stat.text.strip() for stat in row.findAll('td')] for row in data]
    # add player name as first entry in each row
    for i in range(len(player_stats)):
        # ignore header with i+1
        player_stats[i].insert(0, player_names[i+1])
    
    # create player stats dataframe
    player_box_df = pd.DataFrame(player_stats, columns=headerlist)
    # drop 'Reserves' row
    player_box_df.drop(player_box_df[player_box_df['Starters'] == 'Reserves'].index, inplace=True)
    
    # add game id column
    player_box_df.insert(loc=0, column='game_id', value=game_id)
    
    # create team stats dataframe from last row in player stats
    team_box_df = pd.DataFrame(player_box_df.iloc[-1]).T
    
    #drop team totals from player stats df
    player_box_df = player_box_df[:-1].rename(columns={'Starters': 'player'})

    return player_box_df, team_box_df

In [7]:
def merge_boxscores(boxscore_list, team_ids, scope):

    # create tuple for every 2 boxscores in list
    pairs = [((boxscore_list[i]), (boxscore_list[i + 1])) for i in range(0, len(boxscore_list), 2)]
    
    clean_boxscores= []
    
    for pair in pairs:
        
        # combine regular and adv boxscores
        df = pd.concat([*pair], axis=1)
        # drop columns with duplicate names
        df = df.loc[:,~df.columns.duplicated()].copy()
        
        clean_boxscores.append(df)
    
    for i in range(len(clean_boxscores)):
        
        if scope=='team':
            clean_boxscores[i].rename(columns={'Starters': 'team'}, inplace=True)
            clean_boxscores[i]['team'] = team_ids[i]
            
        elif scope=='player':
            clean_boxscores[i].insert(loc=2, column='team', value=team_ids[i])
    
    return clean_boxscores

In [8]:
def change_dtypes(df, num_columns):

    df.replace(to_replace='', value='-99', inplace=True)
    
    for column in num_columns:
        df[column] = df[column].astype('float64')
        
    df.replace(to_replace=-99, value=np.nan, inplace=True)
    
    return df

In [9]:
def create_PIE(player_boxes, totals):
    
    PIE_denom = (totals['PTS'] + totals['FG'] + totals['FT'] - totals['FGA'] - totals['FTA'] + totals['DRB'] + (0.5*totals['ORB']) + totals['AST'] + totals['STL'] + (0.5*totals['BLK']) - totals['PF'] - totals['TOV'])
    player_boxes['PIE'] = round((100 * (player_boxes['PTS'] + player_boxes['FG'] + player_boxes['FT'] - player_boxes['FGA'] - player_boxes['FTA'] + player_boxes['DRB'] + (0.5*player_boxes['ORB']) + player_boxes['AST'] + player_boxes['STL'] + (0.5*player_boxes['BLK']) - player_boxes['PF'] - player_boxes['TOV']) / PIE_denom), 1)
    
    return player_boxes

In [10]:
# connect to sql database
conn = sqlite3.connect('NBA-Game-Database-temp')
driver = webdriver.Chrome(ChromeDriverManager().install())

In [13]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Initialize the WebDriver once at the start
driver = webdriver.Chrome(ChromeDriverManager().install())

for i in range(len(precovid_seasons)):
    
    season_id = precovid_seasons[i]
    season_gamecount = 1
    start_url = 'https://www.basketball-reference.com/leagues/NBA_' + precovid_url_years[i] + '_games.html'
    
    # Open the season schedule page
    driver.get(start_url)
    time.sleep(2)  # delay to wait for page load
    src = driver.page_source
    
    # Create BeautifulSoup object from the page source
    parser = BeautifulSoup(src, 'lxml')
    
    # Every month from the season
    months = parser.find('div', attrs={'class': 'filter'})
    
    if not months:
        print(f"Months filter not found for season {season_id}. Skipping this season.")
        continue
    
    # Partial URLs for each month
    links = months.findAll('a')
    month_links = [base_url + link['href'] for link in links]
    # Only include regular season months (Oct-Apr)
    month_links = month_links[0:7]
    
    for month_url in month_links:
        
        # Navigate to the month URL
        driver.get(month_url)
        time.sleep(2)  # delay to wait for page load
        
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')
        table = parser.find('div', attrs={'class': 'table_container is_setup'})
        
        if not table:
            print(f"Table container not found at {month_url}. Skipping this month.")
            continue
        
        # Check if final month (Apr). If true, set limit for game URLs before playoffs start
        row_num = None
        splits = table.findAll('tr', attrs={'class': 'thead'})
        
        for split in splits:
            if 'Playoffs' in split.text:
                row_num = int(split['data-row'])
                
        # Get partial URLs of every game in the month (if Apr, stop before playoffs)
        if row_num is None:
            game_partial_urls = table.findAll('td', attrs={'class': 'center', 'data-stat': 'box_score_text'})
        else:
            game_partial_urls = table.findAll('td', attrs={'class': 'center', 'data-stat': 'box_score_text'}, limit=row_num)
        
        game_urls = [base_url + url.a['href'] for url in game_partial_urls]
        
        # Open every game URL, retrieve and manipulate data, add to SQL database
        for game_url in game_urls:
    
            driver.get(game_url)
            time.sleep(2)  # delay to wait for page load
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')
            
            # Game info database:
            id_table = parser.find('table', attrs={'class': 'suppress_all stats_table', 'id': 'line_score'})
            
            if not id_table:
                print(f"Line score table not found at {game_url}. Skipping this game.")
                continue
            
            game_info = create_game_info(url=game_url,
                                         season_id=season_id,
                                         season_gamecount=season_gamecount)
            # Will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # Will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]
            
            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # Write game info to SQL database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # Team/player databases:
            
            # 4 boxscore tables: away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs={'class': 'sortable stats_table now_sortable'})
            
            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # Create team and player boxscores
            for idx, stat_table in enumerate(stat_tables):
                # Split player and team boxscores
                player_box_list[idx], team_box_list[idx] = create_boxscores(stat_table, game_id=game_id)
            
            # Team stats database:
            
            # Combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # Prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # Write to SQL database
            team_boxes.to_sql('team_stats', con=conn, if_exists='append', index=False)
            
            # Player stats database:
            
            # Combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # Prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # Create team totals for PIE calculation
            totals = dict(team_boxes.loc[:, 'FG':'PTS'].sum())
            # Add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # Write to SQL database
            player_boxes.to_sql('player_stats', con=conn, if_exists='append', index=False)

            # Increase game count to create next game_id
            season_gamecount += 1

# Close the browser once all scraping is done
driver.quit()
